<a href="https://colab.research.google.com/github/mahikshith/AWS_Sagemaker_ML_trainer/blob/main/AWS_Sagenaker_mL_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Objective :This project explores how to connect jupyter to sagemaker and S3 and train a ML model on sagemaker by extracting data from S3.**

We use AWS CLI to configure and connect to sagemaker rather than using sagemaker UI to create training job , endpoint etc..

Inshort we are just using AWS UI for creating  IAM role , S3 bucket and the best part is we don't have to use sagemaker UI for training and inferencing from end-point

In [3]:
pip install sagemaker  boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.0/404.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import sagemaker
import boto3

sagemaker_client=boto3.client("sagemaker")

""" Boto3 think of it like a bridge between ur python script and AWS
sagemaker" is a string argument passed to the client() function,
telling boto3 that you want to work with the Amazon SageMaker service."""

session=sagemaker.Session()
region=session.boto_session.region_name

# Configure AWS from terminal and using IAM from AWS account to get the region

bucket="mykee65sage"  # s3 bucket name
print(bucket)
print(region)


In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("/content/mob_price_classification_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
df.shape

(2000, 21)

In [6]:
df.isnull().sum()

,0
battery_power,0
blue,0
clock_speed,0
dual_sim,0
fc,0
four_g,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [7]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [9]:
df["price_range"].value_counts()

,count
price_range,
1,500
2,500
3,500
0,500


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [11]:
df.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [12]:
x=df.drop("price_range",axis=1)
y=df["price_range"]

In [14]:
# price_range is the target

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'],
      dtype='object')

In [15]:
# train-test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [17]:
print(x_train.shape),print(x_test.shape)

(1600, 20)
(400, 20)


(None, None)

In [19]:
# need to upload data to s3

train_df = pd.concat([x_train,y_train],axis=1)
test_df = pd.concat([x_test,y_test],axis=1)

In [21]:
# Storing in csv then to s3
train_df.to_csv("train.csv",index=False)
test_df.to_csv("test.csv",index=False)

In [ ]:
# 1. Create a bucket in S3  2. upload the data in to a folder in s3

folder = "sagemaker/mob_classification"

train_path = session.upload_data(path="train.csv",key_prefix=folder,bucket = bucket)
test_path = session.upload_data(path="test.csv",key_prefix=folder,bucket = bucket)



train & test path :

s3://mykee65sage/sagemaker/mob_classification/sklearncontainer/train.csv

s3://mykee65sage/sagemaker/mob_classification/sklearncontainer/test.csv

## **Script** :

"""Imagine you have a text editor open, and you type some code into it.
This line is like clicking "Save As" and naming the file "sagemaker_script_ml.py".
Everything you write in the cell after this line will be saved into that file."""

In [22]:
%%writefile sagemaker_script_ml.py

# random forest classifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import argparse
import os
import joblib
import boto3
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

# custome script :

def function_for_model(model_dr):
  classifier = joblib.load(os.path.join(model_dr,"model.joblib"))
  # loads the model from joblib model

if __name__ == "__main__":

  # Arguements needs for sklearn-sagemaker
  # Hyper-parmeters
  parser = argparse.ArgumentParser()
  parser.add_argument("--n_estimators",type=int,default=110)
  parser.add_argument("--random_state",type=int,default=42)

  parser.add_argument("--max_depth",type=int,default=15)
  parser.add_argument("--n_jobs",type=int,default=-1)

  # data , model , output needs to stored in sagemaker environment

  parser.add_argument("--model-dir",type=str,default=os.environ.get("SM_model_trained_DIR"))
  parser.add_argument("--train",type=str,default=os.environ.get("SM_train"))

  parser.add_argument("--test",type=str,default=os.environ.get("SM_test"))
  parser.add_argument("--train-file",type=str,default="train.csv")
  parser.add_argument("--test-file",type=str,default="test.csv")

  args, _ = parser.parse_known_args()

  train_df = pd.read_csv(os.path.join(args.train,args.train_file))
  test_df = pd.read_csv(os.path.join(args.test,args.test_file))

  x_train = train_df.drop("price_range",axis=1)
  y_train = train_df["price_range"]
  x_test = test_df.drop("price_range",axis=1)
  y_test = test_df["price_range"]

  print("x_train shape : ",x_train.shape)
  print("x_test shape : ",x_test.shape)

  # training :

  model = RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 max_depth=args.max_depth,n_jobs=args.n_jobs)
  model.fit(x_train,y_train)

  model_path = os.path.join(args.model_dir,"model.joblib")
  # joblib - just like pickle - serialized format
  joblib.dump(model,model_path)

  # prediction :
  y_pred = model.predict(x_test)
  test_accuracy = accuracy_score(y_test,y_pred)
  print("test accuracy : ",test_accuracy)
  print(classification_report(y_test,y_pred))
  print()
  print(confusion_matrix(y_test,y_pred))


Writing sagemaker_script_ml.py


## Sagemaker needs to execute the script :

configuration for  SageMaker estimator for training a scikit-learn model :

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "1.2-1" # latest version for sklearn sagmaker sdk

# role="arn:aws:iam::788374821620:role/sagemakeraccess":
# This is the Amazon Resource Name (ARN) of an IAM role
# that SageMaker will use to access resources like S3.
# This role needs necessary permissions for training and deployment.

sklearn_estimator = SKLearn(
    entry_point="sagemaker_script_ml.py",
    role="arn:aws:iam::788374821620:role/sagemakeraccess",
    instance_count=1, # no.of EC2 instances
    instance_type="ml.m5.large", # instance type
    framework_version=FRAMEWORK_VERSION,
    base_job_name="mykee_rf_custom",
    hyperparameters={
        "n_estimators":110,
        "random_state":42,
        "max_depth":15,
        "n_jobs":-1
    },
    use_spot_instances=True,
    max_wait=60*60*4,
    max_run=60*60*2
)


## **Training job launch** :

In [ ]:

sklearn_estimator.fit({"train":train_path,"test":test_path},wait =True)

**Model has been trained under job-name ,let's get the model from s3 for end-point deployment :**

In [ ]:

artifact = sagemaker_client.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

# entire model info is stored in artifact

## End-point deployment for inference :

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "mykee-rf-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::788374821620:role/sagemakeraccess",
    entry_point="sagemaker_script_ml.py",
    framework_version=FRAMEWORK_VERSION,
)


In [ ]:
endpoint_name = "mykee-rf-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

pred = model.deploy(
    initial_instance_count=1, # total EC2 instances
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
)

In [25]:
y_test.value_counts()

,count
price_range,
3,112
0,105
2,92
1,91


## Testing the endpoint for prediction :

In [29]:
x_test.sample(4)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
1924,1982,1,1.6,0,2,1,12,0.2,80,5,20,287,593,1824,13,3,14,1,1,1
544,707,0,0.5,0,0,1,16,0.1,194,2,0,1027,1508,1853,17,14,12,1,1,1
203,671,0,0.9,1,10,0,30,0.7,105,7,17,852,1182,2504,6,4,15,1,0,0
1713,1083,1,1.4,1,2,1,49,0.8,156,5,9,295,503,1338,16,8,5,1,1,1


In [ ]:
print(pred.predict(x_test.sample(4).values.tolist()))

[2,1,3,0]

In [ ]:
# Stop the endpoint :
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)